In [7]:
import pandas as pd
import time
import lxml
import requests
import numpy as np
import datetime
import requests
from bs4 import BeautifulSoup
import pandas.tseries.offsets as offsets

In [8]:
url = 'https://reserve.489ban.net/client/hana-mura/0/plan/availability/room/stay?date=2021-04-06'
pd.read_html(url)

[   4/6  火  7  水  8  木  9  金  10  土  11  日  12  月  13  火  14  水  15  木  16  金  \
 0     NaN   NaN   NaN   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
 
    17  土  18  日  19  月  
 0    NaN    NaN    NaN  ,
    4/6  火  7  水  8  木  9  金  10  土  11  日  12  月  13  火  14  水  15  木  16  金  \
 0     NaN   NaN   NaN   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
 
    17  土  18  日  19  月  
 0    NaN    NaN    NaN  ,
    4/6  火  7  水  8  木  9  金  10  土  11  日  12  月  13  火  14  水  15  木  16  金  \
 0     NaN   NaN   NaN   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
 
    17  土  18  日  19  月  
 0    NaN    NaN    NaN  ,
    4/6  火  7  水  8  木  9  金  10  土  11  日  12  月  13  火  14  水  15  木  16  金  \
 0     NaN   NaN   NaN   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
 
    17  土  18  日  19  月  
 0    NaN    NaN    NaN  ,
    4/6  火  7  水  8  木  9  金  10  土  11  日  12  月  13  火  14  水  15  木  16  金  \
 0     NaN   NaN   NaN   NaN      1      1      1  

In [9]:
html = requests.get(url)
html.encoding = 'EUC-JP'
soup = BeautifulSoup(html.content, 'html.parser')


In [10]:
#divの指定
divs = soup.select('tbody div')

#関数定義のコード
def div_to_availability(div):
    if div.find(class_='fa-circle-o'):
        return 'o'
    elif div.find(class_='fa-exclamation-triangle'):
        return 't'
    elif div.find(class_='fa-close'):
        return 'c'
    elif div.find(class_='fa-phone-square'):
        return 's'
    elif div.find(class_='fa-minus'):
        return 'm'
    else:
        return str(div.a.string).strip()

In [11]:
#成形する
availabilities = [div_to_availability(div) for div in divs]
pd.DataFrame(np.split(np.array(availabilities), 7))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,m,c,c,o,o,o,o,o,o,c,o,o,o,c
1,m,c,c,o,t,o,o,o,o,c,t,t,o,c
2,m,c,c,o,t,o,o,t,o,c,o,o,o,c
3,m,c,c,t,c,t,t,t,c,c,t,t,t,c
4,m,c,c,c,1,1,1,c,1,c,1,c,c,c
5,m,c,c,1,c,c,c,c,1,c,1,c,c,c
6,m,c,c,2,c,1,2,2,2,c,1,2,2,c


In [14]:
#サーバーに負荷を掛けないように調整した完成品

stay_date_list = list(pd.date_range(start=datetime.datetime.now(), periods=3,  freq='14D').strftime('%Y-%m-%d'))
def scrape_hotel_vacancy(stay_date_list):    
    hotel_vacancy = {}
    for stay_date in tqdm(stay_date_list):
        try:
            url = 'https://reserve.489ban.net/client/hana-mura/0/plan/availability/room/stay?date=' + stay_date
            html = requests.get(url)
            html.encoding = 'EUC-JP'
            soup = BeautifulSoup(html.content, 'html.parser')
            divs = soup.select('tbody div')
            availabilities = [div_to_availability(div) for div in divs]
            hotel_vacancy[int(stay_date.replace('-',''))] = pd.DataFrame(np.split(np.array(availabilities), 7))
            time.sleep(1) #スリープ時間
        except IndexError:
            continue
        except:
            break      
    return hotel_vacancy